# Imports

Connect your drive, so the results are saved even if colab kicks you off and saves you some time reinnstalling stuff

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
ORIGIN_DIR = "/content/drive/MyDrive/Transfer/GrainBoundaries" #Set to your own local/gdrive directory you want as a workspace. For simplicity, all files read and saved will be in this folder under ordinary use

In [ ]:
#Will need to do this first if you haven't, better yet, you could fork it and clone your own fork in the case this fork is somehow lost,
#but as long as you already connected google drive, simply cloning it will save it to your drive, so not necessary

#!git clone https://github.com/LostCorgi/PyMDT.git

In [ ]:
import os
import sys
sys.path.append(ORIGIN_DIR+"/PyMDT")
from glob import glob
#import numpy as np
#import matplotlib.pyplot as plt

import numpy as np
import scipy
from scipy import optimize
from scipy.optimize import curve_fit

import cv2 as cv


if importing has an issue, you may need to restart the runtime, especially if you newly cloned the PyMDT.

Change drectory to folder containing your PyMDT folder

In [ ]:
%cd $ORIGIN_DIR

/content/drive/MyDrive/Transfer/GrainBoundaries


Now import all the PyMDT dependencies, and several other packages

sympy is used for creating fittable curves as well as representation of mathematical functions

In [ ]:
import PyMDT
#from PyMDT import *
from PyMDT.MDTdeclaration import * #MDTFile as MDT
from PyMDT.MDTfile import *
import matplotlib.pyplot as plt

from scipy.stats import norm
from scipy.signal import convolve2d
from scipy.interpolate import interpn

import pandas as pd

In [ ]:
import sympy as sy
from sympy import tanh

plotly allows interactive plotting

In [ ]:
import plotly

In [ ]:
import plotly.graph_objects as go
from plotly.graph_objects import FigureWidget

<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


Widget Library (Solara)

In [ ]:
!pip -q install solara

In [ ]:
import solara

helps hide some of the more annoying warnings. If you want, you can comment them out and see what the warnings are when you run it, but I don't think they impact the results.

In [1]:
import warnings
warnings.filterwarnings(action='once')
warnings.filterwarnings("ignore", module='pandas.*')
warnings.filterwarnings("ignore", module='importlib.*', append=True)
warnings.filterwarnings("ignore", module='lambdifygenerated.*', append=True)

# MDTF File Processing

## MDTF Processing

### 0. Data Manipulation/Preprocessing

#### Functions

In [ ]:
def plane_normalize(image):
    #Takes a 2D height array, fits a plane to all of it, and subtracts the plane out.
    def planeModel(points, a, b, c):
        #The equation of the plane in function form for fitting
        return points[0]*a+points[1]*b+c

    #Initial guesses for a, b, c
    initialParameters=[1e-3, 1e-3, 1e-3]
    #Generating the x and y points for the heights in the image
    xRange = np.arange(image.shape[1])
    yRange = np.arange(image.shape[0])
    X, Y = np.meshgrid(xRange, yRange)

    fittedParameters, pcov = scipy.optimize.curve_fit(planeModel, [np.ravel(X), np.ravel(Y)], np.ravel(image), p0 = initialParameters)
    ref_plane = planeModel([X, Y], *fittedParameters)
    normData = image-ref_plane
    return normData

### 4. Profile Projection

#### Parameters

In [ ]:
PROFILE_HALF_LENGTH = 100
PROFILE_SEGMENT_LENGTH = .5

PROFILE_COUNTER = 0

#### Functions

In [ ]:
#x1, y1, x2, y2 = target_line['x1'], target_line['y1'], target_line['x2'],target_line['y2']
#halflen is half the distance on the grid, seglen is distance between points
def line_projection(x1, y1, x2, y2):
    #
    #Get perpendicular Orientation
    p_theta = np.arctan2(y2-y1, x2-x1)+np.pi/2

    #p_theta = 0
    #Get base length units om x and y according to rotation
    p_x = np.cos(p_theta)
    p_y = np.sin(p_theta)

    #Convert if in negative to have positive direction
    if p_x < 0:
        p_y = -p_y
        p_x = -p_x

    return p_x, p_y

def create_line(xc, yc, p_x, p_y, prof_halflen = PROFILE_HALF_LENGTH, prof_seglen = PROFILE_SEGMENT_LENGTH):
    #Get Line points
    #seglen is the length of each line segment
    #halflen is the number of segments on each side
    l_x = p_x * np.arange(-prof_halflen, prof_halflen+1)*prof_seglen
    l_y = p_y * np.arange(-prof_halflen, prof_halflen+1)*prof_seglen

    l_x = l_x + xc
    l_y = l_y + yc

    return l_x, l_y

def extract_profile(image, p_x, p_y, dx=1, dy=1):
    #Use unitless x and y dimension (which assumes equal x,y unit length for interpolation)
    x = np.arange(0, image.shape[1]) * dx
    y = np.arange(0, image.shape[0]) * dy

    #inter_img = interpn((x, y), plane_normalize(image), (p_x, p_y))
    profile = interpn((x, y), plane_normalize(image).T, (p_x, p_y))
    return profile
    #profile = inter_img(p_x, p_y)

def recenter(profile, l_x, l_y):
    min_index = np.argmin(profile)
    return l_x[min_index], l_y[min_index]

def profile_domain(linex, liney):
    xvec = linex-linex[0]
    yvec = liney-liney[0]
    linev = np.sqrt(xvec**2 + yvec**2)
    return linev

def to_xy(xvec, yvec):
    tdf = pd.DataFrame()
    tdf['x'] = xvec
    tdf['y'] = yvec
    return tdf

def take_profile(img, target_line, points = [.25, .5, .75], dx=1, dy=1):
    #target_line = ldf.iloc[index]
    global PROFILE_HALF_LENGTH, PROFILE_SEGMENT_LENGTH, PROCESS_FILE_NAME
    line_vector = ((target_line['x2']-target_line['x1']), (target_line['y2']-target_line['y1']))
    #Unit vector of perpendicular line:
    profile_x, profile_y = line_projection(target_line['x1'], target_line['y1'], target_line['x2'],target_line['y2'])

    profiles = []
    lines = []

    for point in points:
        intersection = (target_line['x1']+line_vector[0]*point, target_line['y1']+line_vector[1]*point)

        line_x, line_y = create_line(intersection[0], intersection[1],
                                    dx*profile_x, dy*profile_y,
                                    PROFILE_HALF_LENGTH, PROFILE_SEGMENT_LENGTH)

        profile = extract_profile(img, line_x, line_y, dx=dx, dy=dy)

        c_x, c_y = recenter(profile, line_x, line_y)

        line_x, line_y = create_line(c_x, c_y,
                                    dx*profile_x, dy*profile_y,
                                    PROFILE_HALF_LENGTH, PROFILE_SEGMENT_LENGTH)
        profile = extract_profile(img, line_x, line_y, dx=dx, dy=dy)

        domain = profile_domain(line_x, line_y)

        profile = to_xy(domain, profile)
        profile.index.name = PROCESS_FILE_NAME

        profiles.append(profile)

        lines.append(to_xy(line_x, line_y))

    return profiles, lines


## MDTF Processing GUI

### Directory Manager

#### Global Variable

In [ ]:
PROCESS_PROFILES = []
PROCESS_FILE_NAME = ''

Z_DATA = None
MDTFI = None
UNIT = 'um'
SPACING = (1., 1.)
ACTIVE_LINE = {'x1':0,'x2':0,'y1':0,'y2':0}
PROFILES = []
LINES = []

POW10 = {'m': 1, 'mm':10**(-3), 'um':10**(-6), 'nm':10**(-9)}

#### functions

In [ ]:
def rescale(df, newunit='um'):
    return df/POW10[newunit]

def load_multi_profile(file):
    '''
    Loads a .txt profile file by name into a dataframe
    '''
    global PROCESS_FILE_NAME

    df = pd.read_csv(file, sep="  +", header=None, skiprows=[0,2], index_col=False, engine='python')
    #df = df.T.set_index(0)
    #df['Profiles'] = ['Profile '+str(i//2+1) for i in range(len(df))]
    #for i in range(0,len(df),2):
    #    pd.DataFrame(data = , columns=['x','y'])

    #df = df.set_index('Profiles', append=True)
    #df = df.swaplevel().T
    #df = df.replace('-', np.NaN).astype('float32')
    #df = df.reset_index(drop=True)

    df = df.T.set_index(0)
    profiles = []
    for i in range(0,len(df),2):
        pdf = df.iloc[i:i+2]
        pdf = pdf[pdf != '-'].dropna(axis=1)
        pdf = pdf.astype('float32').T.reset_index(drop=True)
        pdf.index.name = (os.path.basename(file))
        profiles.append(pdf)

    return profiles #df

def MDTF_update(mdtfi):
    global Z_DATA, SPACING, MDTFI, UNIT
    MDTFI = mdtfi
    SPACING = mdtfi.dimensions[0]['scale'], mdtfi.dimensions[1]['scale'], mdtfi.mesurands[0]['scale']
    unit = mdtfi.mesurands[0]['unit']
    if unit == '':
        unit ='um'
    ratio = POW10[unit]/POW10[UNIT]
    image = mdtfi.data
    Z_DATA = image*ratio*SPACING[2] #raw mdtf
    #UNIT = mdtfi.dimensions_unit

def loadData(filepath, index=0):
    #load mdtf and metadata
    mdtf = MDTFile(filepath)
    #image = mdtf[index].data
    #root = mdtf[index].metadata
    #xml = etree.fromstring(root.encode('utf-16'))
    return mdtf[index] #image, xml
def parseMetaData(xml):
    #parse metadata for desired elements
    #scan_element = xml.getchildren()[1].getchildren()[5]
    #number of points
    x_pt, y_pt = int(xml.findall('.//scshXPoints')[0].text), int(xml.findall('.//scshXPoints')[0].text) #int(scan_element[15].text), int(scan_element[16].text)

    #Spacing (stored in units, MDTF_update shows preferred method to get these quantities)
    x_space, y_space = float(xml.findall('.//scshXSize')[0].text), float(xml.findall('.//scshYSize')[0].text) #float(scan_element[11].text), float(scan_element[12].text)

    return (x_pt, y_pt), (x_space, y_space)

#### Widget

In [ ]:
from pathlib import Path
from typing import Optional, cast

@solara.component
def DirectoryManager(file_loader):
    global PROCESS_FILE_NAME
    file, set_file = solara.use_state(cast(Optional[Path], None))
    path, set_path = solara.use_state(cast(Optional[Path], None))
    directory, set_directory = solara.use_state(Path(ORIGIN_DIR).expanduser())

    with solara.VBox() as main:
        #can_select = solara.ui_checkbox("Enable select")
        '''
        def reset_path():
            set_path(None)
            set_file(None)
        '''
        # reset path and file when can_select changes
        #solara.use_memo(reset_path, [can_select])

        solara.Markdown(f'''
                         # Selections
                         You are in directory: {directory}

                         You selected path: {path}

                         ---

                         You opened file: **{file}**

                         ## File Browser
                         ''')

        solara.FileBrowser(directory,
                           on_directory_change=set_directory, on_path_select=set_path, on_file_open=set_file, #set_file,
                           can_select=True, directory_first=True,
                           filter=lambda x:(x.is_dir() or (x.suffix == '.mdt') or (x.suffix == '.txt')))

        file_loader(file)

        #solara.Info(f"You selected path: {path}")
        #solara.Info(f"You opened file: {file}")
    if file != None:
        PROCESS_FILE_NAME = file.name
    return main

### Plotly Figures

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px

#### Surface Plot

In [ ]:
def custom_surface():
    fig = go.Figure()

    camera = dict(
        up=dict(x=0, y=1, z=0),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=0.0, y=0.0, z=1.)
    )

    fig.update_layout(title='Heightmap', autosize=False,
                    scene_camera=camera,#scene_camera_eye=dict(x=0.0, y=0.0, z=2.00),
                    margin=dict(l=0, r=0, b=0, t=100),
                    width=450, height=500
    )

    #fig.update_yaxes(automargin=True)
    #fig.update_xaxes(automargin=True)
    fig.update_scenes(camera_projection_type='orthographic')


    fig.update_layout(coloraxis_colorbar=dict(
                                            title="Height",
                                            thicknessmode="fraction", thickness=0.02,
                                            lenmode="fraction", len=0.95,
                                            yanchor="top", y=1,
                                            ticks="outside", ticksuffix=" um?"
                                            ),
                    scene_aspectmode='manual',
                    scene_aspectratio=dict(x=1, y=1, z=0.1)
                    )

    #fig.add_trace(go.Surface(z=z_data, opacity=1.0, coloraxis = "coloraxis"))

    #fig.update_traces(contours_z=dict(show=True, usecolormap=False,
    #                                  highlightcolor="limegreen", project_z=True))

    fig.update_layout(coloraxis=dict(colorscale='Plasma'))

    fig.add_trace(go.Surface(z=np.zeros((100, 100)), opacity=1.0, coloraxis = "coloraxis"))

    return fig
#fig.show()

def update_surface(figure, z_data, spacing=(1,1)):
    figure.update_traces(dict(z=z_data, dx=spacing[0], dy=spacing[1], x0=0,y0=0), elector = dict(type="surface"))
    '''
    if len(figure.data) > 0:
        #figure.data[0]['z'] = z_data
        figure.update_traces(dict(z=z_data))
    else:
        figure.add_trace(go.Surface(z=z_data, opacity=1.0, coloraxis = "coloraxis"))
    '''


#### Heatmap

In [ ]:
def custom_heatmap2D():

    fig = go.Figure(data=go.Heatmap(z=np.zeros((3,3)),
                                    colorbar=dict(
                                            title="Height",
                                            thicknessmode="fraction", thickness=0.02,
                                            lenmode="fraction", len=0.95,
                                            yanchor="top", y=1,
                                            ticks="outside", ticksuffix=" um?"
                                            )))

    fig.update_layout(
                      coloraxis=dict(colorscale='Plasma'), width=800,height=780,
                      xaxis=dict(scaleanchor='y', constrain='domain')
                    )

    #hm =
    #fig2.add_trace(hm.data[0])

    fig.add_trace(go.Scatter(
                                x=[None],
                                y=[None],
                                mode="lines+markers+text",
                                name="lines",
                                #text=[],
                                textposition="top center"
                            ))

    fig.add_trace(go.Scatter(
                                x=[None],
                                y=[None],
                                text=[None],
                                mode="lines+markers+text",
                                name="markers",
                                textposition="top center"
                            ))

    return fig
    #fig2.show()

def update_heatmap2D(figure, z_data, spacing=(1,1)):
    #.data[0]
    #hm.update_layout({'coloraxis':"coloraxis"})
    #if len(figure.data) > 0:
        #figure.data[0]['z'] = z_data

    figure.update_traces(dict(z=z_data, dx=spacing[0], dy=spacing[1]), selector = dict(type="heatmap"))

    #else:
    #    hm = px.imshow(z_data, origin='lower')
    #    figure.add_trace(hm.data[0])
    #    figure.update_layout(hm.layout)
'''def mark_heatmap(figure, lines):
    markers_x = []
    markers_y = []
    for line in lines:
        markers_x.append(line['x'].values[0])
        markers_x.append(line['x'].values[-1])
        markers_x.append(None)
        markers_y.append(line['y'].values[0])
        markers_y.append(line['y'].values[-1])
        markers_y.append(None)

    figure.for_each_trace(
        lambda trace: trace.update(x=list(trace['x'])+markers_x,y=list(trace['y'])+markers_y) if trace.name == "markers" else (),
    )'''

def mark_heatmap(figure, line, index):
    markers_x = []
    markers_y = []
    marker_text=[]
    #for line in lines:
    markers_x.append(line['x'].values[0])
    markers_x.append(line['x'].values[-1])
    markers_x.append(None)
    markers_y.append(line['y'].values[0])
    markers_y.append(line['y'].values[-1])
    markers_y.append(None)
    marker_text.append(str(index)+'a')
    marker_text.append(str(index)+'b')
    marker_text.append('')

    figure.for_each_trace(
        lambda trace: trace.update(x=list(trace['x'])+markers_x,y=list(trace['y'])+markers_y,text=list(trace['text'])+marker_text) if trace.name == "markers" else (),
    )

def clear_markers(figure):
    #first figure is heatmap, second is selection line
    figure.data = figure.data[:2]

In [ ]:
def plot_profile(title="profile"):
    fig = go.Figure()

    fig.update_layout(
    title=title,
    xaxis_title="Profile Length",
    yaxis_title="Height",
    legend_title="Legend"
    )

    '''fig.add_trace(go.Scatter(
                            x=[0],
                            y=[0],
                            mode="lines",
                            name=f"lines"
                            ))'''
    return fig

def update_profile(figure, profiles):
    #global PROCESS_PROFILES
    figure.data = []
    for i, profile in enumerate(profiles):
        figure.add_trace(go.Scatter(
                                    x=profile['x'],
                                    y=profile['y'],
                                    mode="lines",
                                    name=f"{profile.index.name}_{i}"
                                    )
                        )
    #figure.update_traces(dict(z=z_data, dx=spacing[0], dy=spacing[1], x0=0,y0=0), selector = dict(type="heatmap"))


#### Plotly Figure Instances (global variables)

Basically, fig1 is a 3d heatmap, so you can examine the surface in 3d. It does scale down z-contrast by 10 so that you can recognize details in fig2 in fig1. Currently I do not use fig1 however, but you may call copy_to_surface() after plotting in the heatmap if you want to look at it, but it can take some time to render, and the heatmap already takes long enough, so I decided not to make it automatically occur.

fig_preview an fig3 are both profile plots. Preview is for those using the MDT file processing to take a profile and look at the profile. If you like it, you can send it to fig3 (processing). If you're using .txt profiles, they will open straight into fig3. Anything that is visible in fig3 (sometimes things didn't show for some reason, but I think it's fixed) will be sent to be processed. The text printed below the widget may indicate if anything went wrong. If everything goes right, then the final widget will appear with a table of results and fitted plots.

In [ ]:
fig1 = go.FigureWidget(data=custom_surface())
fig2 = go.FigureWidget(data=custom_heatmap2D())
fig_preview = go.FigureWidget(data=plot_profile("Previewing"))
fig3 = go.FigureWidget(data=plot_profile("Processing"))
#fw = go.FigureWidget(data=fig.data)#.element()

##### Forward to Plots or Between

In [ ]:
def copy_to_surface():
    global fig1, fig2
    #fig.data[0]['z'] =
    shape = fig2.data[0]['z'].shape
    x = fig2.data[0]['dx'] * np.arange(shape[0])
    y = fig2.data[0]['dy'] * np.arange(shape[1])
    fig1.update_traces(dict(z=fig2.data[0]['z'], x=x, y=y))

In [ ]:
def send2heatmap():
    global fig2, Z_DATA, SPACING, MDTFI, UNIT
    #image = mdtfi.data
    #Z_DATA = image #raw mdtf
    #print('normalized')
    #update_surface(fig, z_data)
    #print('updated surface plot')
    #print('sbs')
    #SPACING = mdtfi.dimensions[0]['scale'], mdtf[0].dimensions[1]['scale'], mdtf[0].mesurands[0]['scale']
    #MDTFI = mdtfi#parseMetaData(xml)
    #UNIT = mdtfi.dimensions_unit

    z_data = plane_normalize(Z_DATA) #normalized mdtf
    update_heatmap2D(fig2, z_data, SPACING)

    #print('updated heatmap')


In [ ]:
def send2preview():
    global fig_preview, PROFILES
    update_profile(fig_preview, PROFILES)

In [ ]:
def send2process():
    global fig3, PROCESS_PROFILES, fig2, LINES, PROFILES
    for i, profile in enumerate(PROFILES):
        profile.index.name += f"_{i}"
        PROCESS_PROFILES.append(profile)
        index = len(PROCESS_PROFILES)
        mark_heatmap(fig2, LINES[i], index)

    LINES = []
    PROFILES = []
    #print('before profiles')
    update_profile(fig_preview, PROFILES)
    #print('before process')
    update_profile(fig3, PROCESS_PROFILES)
    #print('completed')
    #append_markers()



In [ ]:
def txt2process():
    global fig3, PROCESS_PROFILES
    update_profile(fig3, PROCESS_PROFILES)


#### Profile Extraction

In [ ]:
# create our callback function
def update_point(trace, points, selector):
    '''Used as callback for placing points on the heatmap'''
    global fig2
    #for i in points.point_inds:
    #coord = points.point_inds[0]
    xs = points.xs[0]
    ys = points.ys[0]
    origx = fig2.data[1]['x'][0]
    origy = fig2.data[1]['y'][0]
    with fig2.batch_update():
        fig2.data[1]['x'] = [xs, origx]
        fig2.data[1]['y'] = [ys, origy]
    #    #with f.batch_update():
    #    #    scatter.marker.color = c
    #    #    scatter.marker.size = s

In [ ]:
def get_line():
    '''Updates ACTIVE_LINE using the line from the heatmap'''
    global ACTIVE_LINE, fig2, SPACING #, sc

    #preprocess(Z_DATA, filter=FILTER_TYPE, method=KUWAHARA_METHOD, radius=KERNEL_SIZE)

    ACTIVE_LINE['x1'] = fig2.data[1]['x'][0]
    ACTIVE_LINE['y1'] = fig2.data[1]['y'][0]
    ACTIVE_LINE['x2'] = fig2.data[1]['x'][1]
    ACTIVE_LINE['y2'] = fig2.data[1]['y'][1]

def collect_profiles(points = [.25, .5, .75]):
    global Z_DATA, ACTIVE_LINE, PROFILES, LINES
    get_line()
    preprocessed = plane_normalize(Z_DATA)
    #Next Steps
    PROFILES, LINES = take_profile(preprocessed, ACTIVE_LINE, points=points, dx=SPACING[0], dy=SPACING[1])
    send2preview()

    #fig, axs = plt.subplots(1, 3, figsize=(10, 8))

    #axs[0].plot(profiles[0])
    #axs[1].plot(profiles[1])
    #axs[2].plot(profiles[2])

    #Add interaction step

    #Automatically process fitting of plots

    #If data is kept, markings put onto plot

    #Finally Save CSV

#### Widget Display and Linking

#### Assign callback to fig2(Heatmap)

In [ ]:
fig2.data[0].on_click(update_point)

#### Internal components of the Directory Manager, handles opening of both .txt and .mdt files

In [ ]:

def open_txt_file(file):
    global PROCESS_FILE_NAME, PROCESS_PROFILES, UNIT
    profiles = load_multi_profile(file.as_posix())
    rescaled = [rescale(profile, UNIT) for profile in profiles]
    PROCESS_PROFILES += rescaled
    txt2process()
    #for profile in profiles:
    #    PROCESS_PROFILES.append(rescale(profile, UNIT))

In [ ]:
@solara.component
def MDTF_Loader(mdtf):
    #mdtf, set_mdtf = solara.use_state(cast(Optional[MDTFile],None))
    max_index, set_max_index = solara.use_state(0)
    index, set_index = solara.use_state(0)
    is_loading, set_is_loading = solara.use_state(False)

    if mdtf != None:
        set_max_index(len(mdtf))
    else:
        set_max_index(0)

    def click_callback():
        MDTF_update(mdtf[index])
        set_is_loading(True)
        send2heatmap()
        set_is_loading(False)
    with solara.VBox() as main:
        solara.SliderInt("Index", value=index, min=0, max=max_index-1, on_value=set_index)
        solara.Markdown(f"**Int value**: {index}")
        solara.ProgressLinear(is_loading)
        solara.Button("Load .mdt File", on_click=click_callback)
    return main

@solara.component
def FileLoader(file):

    if (file != None) and (file.suffix == '.txt'):
        open_txt_file(file)

    with solara.HBox() as main:
        if (file != None) and (file.suffix == '.mdt'):
            with solara.VBox():
                MDTF_Loader(MDTFile(file.as_posix()))
                solara.Button("Take Profile (preview)", on_click=collect_profiles)
            solara.Button("Send to Process", on_click=send2process)
    return main

#### Directory Manager Prompt

In [ ]:
PROCESS_PROFILES = []
PROCESS_FILE_NAME = ''

Z_DATA = None
MDTFI = None
UNIT = 'um'
SPACING = (1., 1.)
ACTIVE_LINE = {'x1':0,'x2':0,'y1':0,'y2':0}
PROFILES = []
LINES = []

fig1 = go.FigureWidget(data=custom_surface())
fig2 = go.FigureWidget(data=custom_heatmap2D())
fig_preview = go.FigureWidget(data=plot_profile("Previewing"))
fig3 = go.FigureWidget(data=plot_profile("Processing"))
#fw = go.FigureWidget(data=fig.data)#.element()
fig2.data[0].on_click(update_point)

In [ ]:
@solara.component
def PlotlyFigure(dm, fig1, fig2, fig3, fig_preview):
    #w = FigureWidget(data=fig.data)
    mdt_plots = solara.HBox(children=[fig2, fig1])
    analysis = solara.HBox(children=[fig_preview, fig3])
    vb = solara.VBox(children=[mdt_plots, dm, analysis])
    return vb

#### Demo the Directory Manager by itself

In [ ]:
dm = DirectoryManager(FileLoader)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

PlotlyFigure(dm, fig1, fig2, fig3, fig_preview)

<IPython.core.display.Javascript object>

Html(layout=None, style_='display: none', tag='span')

Cannot show ipywidgets in text

# Bulk Process Profiles

## Tip Definition

In [ ]:
def make_tip(tip_width = 101e-3, diameter = 20e-3, tip_slope = np.tan(np.deg2rad(70)), x_space = 1e-3, z_space = 1e-3):
    #tip_width represents the total width of the tip, needs to exceed the diameter, which is the width of the curved region of the tip. tip_slope describes the steepness of the tip.
    #the x and z space terms describe the common unit of measure

    radius = diameter/2
    tip_points = tip_width/x_space

    rtip_width = diameter/x_space+1
    rx = np.linspace(-radius, radius, num=int(rtip_width), endpoint=True)
    #creates half circle tip
    rtip = radius-np.sqrt(radius**2 - rx**2)

    intercept = radius*(1-tip_slope)
    side_x = np.arange(radius/x_space+1, (tip_points-rtip_width)//2+1)*x_space

    tip = np.concatenate([-tip_slope*-np.flip(side_x)+intercept,
                         rtip,
                         tip_slope*side_x+intercept])

    #tip_x = (np.arange(0, len(tip)) - len(tip)/2)*x_space #np.linspace(-1e-5, 1e-5, 49, dtype='float32')**2
    tip_x = np.concatenate([-np.flip(side_x), rx, side_x])
    tip = tip*(z_space/x_space)#/((B*t)**(.25))
    return tip_x, tip

    #tip_u = tip_x/((B*t)**(.25))

def sample_tip(data, diameter = 20e-3, angle = np.deg2rad(70)):
    #data describes a distance from the tip center
    absdata = np.abs(data)
    radius = diameter/2
    x_cutoff = radius*np.sin(angle)
    y_offset = x_cutoff*np.tan(angle)
    y_shift = radius*np.cos(angle)
    curve_region = absdata<x_cutoff
    absdata[curve_region] = -np.sqrt(radius**2 - absdata[curve_region]**2) + y_shift
    absdata[np.invert(curve_region)] = (absdata[np.invert(curve_region)])*np.tan(angle)-y_offset
    return absdata

def sample_tip_function(diameter = 20e-3, angle = np.deg2rad(70)):
    return lambda x: sample_tip(x, diameter, angle)

def tip_offset_generator(xvals, tip_function=sample_tip_function()):
    #What we do is treat each x value in xvals as a new possible center.
    #We start by repeating the same array as many times as there are, so offsets[0] == offsets[1] == xvals
    #There is probably a way to limit the checked region to a certain range of tip points rather than a tip that extends the entire profile width
    #However, this makes it a lot more complicated and loses out on the benefit that we can handle x points that have different spacings or uneven spacings without having to include spacing as another parameter
    #if you have a lot of x values and want to speed up this process by making it more sparse at the expense of accuracy, it's probably doable but hopefully this solution works
    offsets = np.tile(xvals, (xvals.shape[0],1))
    #now each row index in offsets is a value in xvals, which we can subtract, so the first row is offsets[0] - xvals[0], offsets[1]-xvals[1]
    offsets -= xvals[..., np.newaxis]
    #Since we use this directly with the tip, we can apply the tip function to convert the offsets
    offsets = sample_tip(offsets)
    return offsets




In [ ]:
#plt.plot(np.linspace(-.02, .02, 100), sample_tip_function()(np.linspace(-.02, .02, 100)))

## Mullin's Ideal Curve

### Parameters (only used if you want to make mullins curve with get_ideal_profile, not used in app)

In [ ]:
#number of points used in the generation of the mullin's or other fitting curve
MAX_PTS = 1000

#This is an example for get_ideal_profile, a way to make an example ideal profile
x_base = np.arange(-MAX_PTS//2, MAX_PTS//2+1)*1e-9
#Use fittable_ideal_function for a fittable function, function fits on the x variable as input

### Function

In [ ]:
def get_ideal_profile(x_val = x_base, deg=25):
    x, t = sy.symbols('x t')
    m, B = sy.symbols('m B')
    u = x/((B*t)**(1/4))
    #Coefficient expression
    F_co = m*(B*t)**.25
    #Onluse used by get_ideal_profile
    us = sy.symbols('u')

    #Generates a fittable equation based on Mullin's equation using sympy
    #This one assumes the x_values
    coeff = [-np.sqrt(2)/scipy.special.gamma(5/4),
            1, 0, -1/(6*np.sqrt(np.pi))]
    for i in range(4, deg):
        newval = coeff[-4]*(i-5)/(4*(i)*(i-1)*(i-2)*(i-3))
        coeff.append(newval)

    F = 0
    for i in range(deg):
        F += coeff[i]*us**i

    F = F_co*F
    Flam = sy.lambdify(us, F)
    Fv = np.vectorize(Flam)
    u = x_val/((B*t)**(.25))

    ideal = np.concatenate([Fv(-u[:len(u)//2+1]), Fv(u[len(u)//2+1:])])
    return ideal

def fittable_ideal_function(deg = 50):
    '''
    Mullin's equation
    deg is number of terms in the summation
    '''
    x, t = sy.symbols('x t')
    m, B = sy.symbols('m B')
    u = x/((B*t)**(1/4))
    #Coefficient expression
    F_co = m*(B*t)**.25

    #Series coefficients
    coeff = [-np.sqrt(2)/scipy.special.gamma(5/4),
            1, 0, -1/(6*np.sqrt(np.pi))]
    #Create coefficients
    for i in range(4, deg):
        newval = coeff[-4]*(i-5)/(4*(i)*(i-1)*(i-2)*(i-3))
        coeff.append(newval)
    F = 0
    #sum coefficients of series
    for i in range(deg):
        F += coeff[i]*u**i
    F = F_co*F

    lam_F = sy.lambdify([x, m, t, B], F, 'numpy')
    lam_F_v = np.vectorize(lam_F)
    return lam_F_v

In [ ]:
def fittable_ideal_custom(deg, use_n=False):
    #includes a c offset so that the parameter count matches
    default_func = fittable_ideal_function(deg)
    if use_n:
        def fittable(x, m, t, B, n, c):
            return default_func(x, m, t, B) + n*x + c
    else:
        def fittable(x, m, t, B, c):
            return default_func(x, m, t, B) + c
    return fittable

## Fitting

### Functions

Function that turns a slope into a dihedral angle 180-2*(atan(slope))

In [ ]:
def slope_convert(slope):
    return 180-2*np.rad2deg(np.arctan(slope))

In [ ]:
class BoundaryFitter():
    def __init__(self, m = 0.15, t = 3600*4, B = 3.4e-20, n = 0, c = 0, deg = 3, use_n = False):
        #m corresponds to slope, t to time, B to a physical parameter, n and c are an extra nx+c term used to add an offset linear gradient to each profile to potentially improve the fit.

        #May not need to keep these params explicitly around in the class, but as local variables in some functions
        self.m = m #beta

        #Time
        self.t = t
        #B = (vs * Ds * Omega^2 * ys)/(k * T)
        self.B = B
        self.n = n
        self.c = c

        self.deg = deg

        self.use_n = use_n
        #Distance in micrometer
        #self.x_space, self.y_space, self.z_space = 1.0e-3, 1.0e-3, 1.0e-3

        #Unit represents the 10^self.U exponent, which describes the length scale relative to a meter
        #self.U = -6
        self.init()

    def init(self):

        self.ideal = fittable_ideal_function(self.deg)
        self.ideal_check = fittable_ideal_custom(self.deg, self.use_n)

        #tip convolution step
        def tip_conv(x, m, t, B):
            ideal_ = self.ideal(x, m, t, B)
            #Subtracting the offsetted tip at each location gives a max value which
            combined = ideal_-self.offsets
            index = np.argmax(combined, axis=1)
            return ideal_[index]

        if self.use_n:
            def lam_Ft(x, m, t, B, n, c):
                return tip_conv(x, m, t, B)+n*x+c
        else:
            def lam_Ft(x, m, t, B, c):
                return tip_conv(x, m, t, B)+c

        #In creating the curve, constant quantities will be class variables, while x(first parameter) and subsequent are fittable quantities
        self.curve = lam_Ft

    def get_param_list(self):
        if self.use_n:
            return [self.m, self.t, self.B, self.n, self.c]
        else:
            return [self.m, self.t, self.B, self.c]

    def get_param_dict(self):
        if self.use_n:
            return {'m': self.m, 't': self.t, 'B': self.B, 'n': self.n, 'c': self.c}
        else:
            return {'m': self.m, 't': self.t, 'B': self.B, 'c': self.c}

    def fit(self, curve, guesses=None):
        if guesses == None:
            guesses = self.get_param_list()

        #we set this for every curve, since the x axis for each curve may be different, must be set for the tip convolution function
        self.offsets = tip_offset_generator(curve[0])

        parameters, covariances = curve_fit(self.curve, curve[0], curve[1], p0=guesses)
        #if self.use_n:
        #    self.m, self.t, self.B, self.n, self.c = parameters
        #else:
        #    self.m, self.t, self.B, self.c = parameters
        m_std = covariances[0,0] #the diagonals have the std values for each fit quantity, and m is the first fit quantity
        return parameters, m_std

    def check(self, x, params, ideal=False):

        func = self.ideal_check if ideal else self.curve
        if self.use_n:
            self.m, self.t, self.B, self.n, self.c = params
            return func(x, self.m, self.t, self.B, self.n, self.c)
        else:
            self.m, self.t, self.B, self.c = params
            return func(x, self.m, self.t, self.B, self.c)

    def processing(self, df, file):
        #df = load_multi_profile(file)
        #for profile in df.columns.levels[0]:
        #Substitute with below
        #profile = (df.columns.levels[0])[0]

        #plt.clf()
        #ax1 = plt.gca()
        #ax2 = ax1#.twinx()
        if file == None:
            if df.index.name != None:
                file = df.index.name
            else:
                global PROCESS_FILE_NAME
                file = PROCESS_FILE_NAME

        left, right = split_curves(df)

        #ax1.plot(left[0], left[1], 'blue', label='left')
        #ax1.plot(right[0], right[1],'red', label='right')

        mguessl = self.m #(left[1][6] - left[1][0])/(left[0][6] - left[0][0])
        mguessr = self.m #(right[1][6] - right[1][0])/(right[0][6] - right[0][0])
        mid_index = get_mid_index(df['y'].values)
        left_params = rightparams = None
        if self.use_n:
            initial_params = mguessl, self.t, self.B, self.n, df['y'].values[mid_index]
        else:
            initial_params = mguessl, self.t, self.B, df['y'].values[mid_index]

        if self.use_n:
            columns = ['m', 't', 'B', 'n', 'c']
        else:
            columns = ['m', 't', 'B', 'c']

        try:
            params, m_std = self.fit(left, initial_params)
            #ax1.plot(left[0], self.check(left[0]),'c--', label='leftf')
            left_params = pd.DataFrame([params], columns=columns, index=[file]) #bf.get_param_list()
            left_params['angle'] = slope_convert(left_params['m'])
            left_params['m_std'] = m_std

            left_params['x'] = [left[0]]
            left_params['y'] = [left[1]]
            left_params['f'] = [self.check(left[0], params)]
            left_params['i'] = [self.check(left[0], params, ideal=True)]
            #ax2.plot(left[0], self.check(left[0],params,True),'c-.', label='lefti')
        except RuntimeError:
            print('Left side of '+file+' could not be fit, skipping inclusion in param dataframe')
        try:
            params, m_std = self.fit(right, initial_params)
            #ax1.plot(right[0], self.check(right[0]),'m--', label='rightf')
            right_params = pd.DataFrame([params], columns=columns, index=[file])
            right_params['angle'] = slope_convert(right_params['m'])
            right_params['m_std'] = m_std

            right_params['x'] = [right[0]]
            right_params['y'] = [right[1]]
            right_params['f'] = [self.check(right[0], params)]
            right_params['i'] = [self.check(right[0], params, ideal=True)]
            #ax2.plot(right[0], self.check(right[0],params,True),'m-.', label='righti')
        except RuntimeError:
            print('Right side of '+file+' could not be fit,, skipping inclusion in param dataframe')
        #plt.show()
        if isinstance(left_params, pd.DataFrame) and isinstance(right_params, pd.DataFrame):
            a = pd.concat([left_params, right_params], keys=['left', 'right'], axis=1)
            return a

        return None

class PolyFitter():
    def __init__(self, m, c, deg, form = 'poly'):
        #m estimate, c vertical level, deg is degrees+1 (not zero indexing)
        #might be redundant now that we return these quantities
        self.m = m
        self.c = c
        self.deg = deg
        self.form = form
        self.x = sy.symbols('x')
        #self.U = -6
        if deg < 2:
            print('need more degrees if using poly')

        self.reload()

    def reload(self):
        self.ideal = 0
        if self.form == 'poly':
            self.constants = sy.symbols(','.join(['c']+['c'+str(i) for i in range(self.deg-1)]))
            self.ideal += self.constants[0]
            for i, c in enumerate(self.constants):
                self.ideal += c*self.x**(i+1)
        if self.form == 'tanh':
            c = sy.symbols('c,b,a,w')
            self.constants = c
            self.ideal = c[3]*tanh(c[1]*self.x+c[2])+c[0]
        if self.form == 'sine':
            self.constants = sy.symbols(','.join(['c']+['k'+str(i) for i in range(self.deg-2)]+['d']))
            d = self.constants[-1] #used as a common scaling term inside the sine function
            self.ideal += self.constants[0] #the c value
            #F = c + sum: k*sin((n+1)/d x)
            for i, c in enumerate(self.constants[1:-1]):
                self.ideal += c*sy.sin((i+1)/d*self.x)

        self.deriv = sy.lambdify([self.x]+list(self.constants), sy.diff(self.ideal, self.x), 'numpy')
        self.deriv = np.vectorize(self.deriv)

        self.ideal = sy.lambdify([self.x]+list(self.constants), self.ideal, 'numpy')
        self.ideal = np.vectorize(self.ideal)

        def tip_conv(x, *parameters):
            ideal_ = self.ideal(x, *parameters)
            #Subtracting the offsetted tip at each location gives a max value which
            combined = ideal_-self.offsets
            index = np.argmax(combined, axis=1)
            return ideal_[index]

        self.function = tip_conv

    #Had to remove the inverted function fitting so that I could put confidence in the dilation function, as it only makes sense if applied to the surface curve, not the inverted one
    def ccurve(self, curve):
        x = curve[0]
        y = curve[1]
        y2 = -np.flip(y - curve[1][0])+curve[1][0]
        xx = np.concatenate([-np.flip(x), x])
        yy = np.concatenate([y2, y])
        return xx, yy

    def fit(self, curve, guesses=None,csub=None):

        if csub != None:
            self.c = csub
        if guesses == None:
            if self.form == 'tanh':
                guesses = [self.c,self.m,0,1]
            elif self.form == 'sine':
                #this should not behave like adding arrays together, they are being appended like python lists
                guesses = [self.c]+[self.m]*(self.deg-2)+[10]
            else:
                guesses = [self.c,self.m] +[-1]*(self.deg-2)

        #we set this for every curve, since the x axis for each curve may be different, must be set for the tip convolution function
        self.offsets = tip_offset_generator(curve[0])
        #curve = self.ccurve(curve0)
        parameters, covariances = curve_fit(self.function, curve[0], curve[1], p0=guesses)
        #self.m, self.t, self.B, self.c = parameters
        m_std = covariances[1,1] #the diagonals have the std values for each fit quantity, and m is the second fit quantity after c
        return parameters, m_std

    def check(self, x, params, ideal=False):
        if ideal:
            self.ideal(x, *params)
        else:
            return self.function(x, *params)

    def get_slope(self, params):
        return self.deriv(0, *params)

    def processing(self, df, file):
        #profiles = load_multi_profile(file)

        #for profile in df.columns.levels[0]:
        #Substitute with below

        #profile = (df.columns.levels[0])[0]

        #plt.clf()
        #ax1 = plt.gca()
        #ax2 = ax1#.twinx()

        #left, right = split_curves(df[profile])
        #l,r = self.ccurve(left), self.ccurve(right)
        if file == None:
            if df.index.name != None:
                file = df.index.name
            else:
                global PROCESS_FILE_NAME
                file = PROCESS_FILE_NAME

        left,right = split_curves(df)
        #ax1.plot(left[0], left[1], 'blue', label='left')
        #ax1.plot(right[0], right[1],'red', label='right')

        #mguessl = (left[1][6] - left[1][0])/(left[0][6] - left[0][0])
        #mguessr = (right[1][6] - right[1][0])/(right[0][6] - right[0][0])
        mid_index = get_mid_index(df['y'].values)
        minval = df['y'].values[mid_index] #*10**(-self.U), already corrected

        left_params = rightparams = None
        #initial_params = mguessl, 0.0001, 0.00005, 0, df[profile,'y'].values[mid_index]*10**(-self.U)
        try:
            params, m_std = self.fit(left, csub=minval)
            #ax1.plot(left[0], self.check(left[0], params),'c--', label='leftf')
            left_params = pd.DataFrame([params], columns=[con.name for con in self.constants],index=[file]) #bf.get_param_list()
            m_p = self.get_slope(params)
            left_params['m'] = m_p
            left_params['angle'] = slope_convert(m_p)
            left_params['m_std'] = m_std
            left_params['x'] = [left[0]]
            left_params['y'] = [left[1]]
            left_params['f'] = [self.check(left[0], params)]
            left_params['i'] = [self.check(left[0], params, ideal=True)]

        except RuntimeError:
            print('Left side of '+file+' could not be fit, skipping inclusion in param dataframe')
        #self.m, self.t, self.B, self.n, self.c = mguessr, 0.0001, 0.00005, 0, df[profile,'y'].values[mid_index]*10**(-self.U)
        try:
            params, m_std = self.fit(right, csub=minval)
            #ax1.plot(right[0], self.check(right[0], params),'m--', label='rightf')
            right_params = pd.DataFrame([params], columns=[con.name for con in self.constants],index=[file])
            m_p = self.get_slope(params)
            right_params['m'] = m_p
            right_params['angle'] = slope_convert(m_p)
            right_params['m_std'] = m_std
            right_params['x'] = [right[0]]
            right_params['y'] = [right[1]]
            right_params['f'] = [self.check(right[0], params)]
            right_params['i'] = [self.check(right[0], params, ideal=True)]

        except RuntimeError:
            print('Right side of '+file+' could not be fit, skipping inclusion in param dataframe')

        #plt.show()
        if isinstance(left_params, pd.DataFrame) and isinstance(right_params, pd.DataFrame):
            a = pd.concat([left_params, right_params], keys=['left', 'right'], axis=1)
            return a

        return None


## Processing and Saving

### Functions

### Preprocess

In [ ]:
def get_mid_index(yvals):
    #Tries to find the midpoint as the minimum value within the middle 40% of datapoints
    yvallen = yvals.shape[0]
    midlow = int(yvallen/2) - int(yvallen/5)
    midhigh = int(yvallen/2) + int(yvallen/5)
    if midhigh == midlow:
        midhigh+=1
    midregion = yvals[midlow:midhigh]
    midpoint_index = midlow + midregion.argmin()
    return midpoint_index

def split_curves(df):
    #Divides grooves into left and right curves
    yvals = df['y'].values
    midpoint_index = get_mid_index(yvals)
    right_curve = -df['x'].values[midpoint_index]+df['x'].values[midpoint_index:], df['y'].values[midpoint_index:]
    left_curve = df['x'].values[:midpoint_index+1], np.flip(df['y'].values[:midpoint_index+1])
    return left_curve, right_curve

def file_filter(directory, filter='*.txt'):
    return glob(filter, root_dir=directory)

## Visualization

### Global

In [ ]:
PROCESSED_DATA = pd.DataFrame() #Holds the actual results in a table format

CONTROL_ELEMENTS = dict(fitter='mullins', deg=25, m=.14, t= 1000, B=10, c=0, n=0, use_n=False, form='tanh') #Default values for the fitting algorithms

TABLE = go.FigureWidget() #Holds the representation widget that summarizes results as a table and plots

### Functions

Creates a figure and updates the TABLE global variable with the new figure of a table plus subplots

In [ ]:
from plotly.subplots import make_subplots
def make_summary_fig():
    global TABLE, PROCESSED_DATA

    #if df.empty:
    #    TABLE.data = []
    #    return
    df = PROCESSED_DATA
    PROCESSED_FILES = len(df)
    show_columns = [k for k in df.columns[:] if ((k != 'x') or (k != 'y') or (k != 'f') or (k != 'i'))]
    indices = df.index.values
    #print('prep')

    fig = make_subplots(
                        rows=PROCESSED_FILES+1, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.03,
                        specs=[[{"type": "table"}]]+[[{"type": "scatter"}]]*PROCESSED_FILES,
                        subplot_titles=['dataframe']+PROCESSED_DATA.index.values.tolist()
    )


    fig.add_trace(
                    go.Table(
                            header=dict(
                            values = ['Index']+show_columns,
                            font=dict(size=10),
                            align="left"
                        ),
                        cells=dict(values=[indices]+[df[k].tolist() for k in show_columns], align = "left")
                    ),
                    row=1, col=1
    )
    #print('post table')
    for i in range(PROCESSED_FILES):
        row = df.iloc[i]
        name = row.name
        left=dict(type='scatter',
                    x=row[('left','x')],
                    y=row[('left','y')],
                    name='original left',
                    mode='lines',
                    line=dict(color='blue'))
        right=dict(type='scatter',
                    x=row[('right','x')],
                    y=row[('right','y')],
                    name='original right',
                    mode='lines',
                    line=dict(color='red'))

        leftf=dict(type='scatter',
                    x=row[('left','x')],
                    y=row[('left','f')],
                    name='fitted left',
                    mode='lines',
                    line=dict(color='blue', dash='dash'))
        rightf=dict(type='scatter',
                    x=row[('right','x')],
                    y=row[('right','f')],
                    name='fitted right',
                    mode='lines',
                    line=dict(color='red', dash='dash'))

        lefti=dict(type='scatter',
                    x=row[('left','x')],
                    y=row[('left','i')],
                    name='fitted ideal left',
                    mode='lines',
                    line=dict(color='blue', dash='dot'))
        righti=dict(type='scatter',
                    x=row[('right','x')],
                    y=row[('right','i')],
                    name='fitted ideal right',
                    mode='lines',
                    line=dict(color='red', dash='dot'))

        fig.add_trace(left, row=i+2, col=1)
        fig.add_trace(right, row=i+2, col=1)
        fig.add_trace(leftf, row=i+2, col=1)
        fig.add_trace(rightf, row=i+2, col=1)
        fig.add_trace(lefti, row=i+2, col=1)
        fig.add_trace(righti, row=i+2, col=1)

    #print('post loop')

    fig.update_layout(
    height=800,
    showlegend=False,
    title_text="Processed Results",
    )

    TABLE.update(dict1=fig.to_dict(), overwrite=True)
    #TABLE.update(dict1=fig.to_dict(), overwrite=True)
    #return fig #go.FigureWidget(data=fig)



Function that attempts to use the fitters to fit and return a dataframe for each profile that can be concatenated with other dataframes that can be saved.

In [ ]:
def try_process(name=None):
    global PROCESS_PROFILES, CONTROL_ELEMENTS, PROCESSED_DATA
    profiles_ = PROCESS_PROFILES
    fitter = CONTROL_ELEMENTS['fitter']
    deg = CONTROL_ELEMENTS['deg']
    m = CONTROL_ELEMENTS['m']
    t = CONTROL_ELEMENTS['t']
    B = CONTROL_ELEMENTS['B']
    c = CONTROL_ELEMENTS['c']
    use_n = CONTROL_ELEMENTS['use_n']
    form = CONTROL_ELEMENTS['form']

    if len(PROCESS_PROFILES) == 0:
        return

    if fitter == 'mullins':
        fitr = BoundaryFitter(deg=deg, m=m, B=B, t=t, c=c, use_n=use_n)
    else:
        fitr = PolyFitter(m=m, c=c, deg=deg, form=form)

    processed_ = [fitr.processing(p, name) for p in profiles_]
    processed_ = [p for p in processed_ if isinstance(p, pd.DataFrame)]
    if len(processed_) > 0:
        processed = pd.concat(processed_)
        PROCESSED_DATA = processed
        print(PROCESSED_DATA.loc(axis=1)[:, 'angle'].describe())
    else:
        print('no files processed?')

    #summary_fig = make_summary_fig(processed)

    #return summary_fig

Save the dataframe PROCESSED_DATA and clears the PROCESS_PROFILES, this way when new profiles are appended they don't result in duplicates

In [ ]:
def save_data(name):
    global PROCESSED_DATA, PROCESS_PROFILES, fig3
    PROCESSED_DATA.to_csv(name)
    PROCESS_PROFILES = []
    update_profile(fig3, PROCESS_PROFILES)

In [ ]:
'''
#Probably can delete, was thinking of ways to save into multiple csv for better organization, but it looks like saving entire arrats in a cell is possible.
PROCESSED_DATA.dtypes
np.setdiff1d(PROCESSED_DATA.columns.levels[1], ['x', 'y', 'f', 'i']) #.difference(['left'])
PROCESSED_DATA.select_dtypes(include=['float']).T.reset_index().to_csv('/content/wtf.csv')
import pandas as pd
import numpy as np
pd.DataFrame(np.zeros((3,3))).to_csv('why.csv')
temp = PROCESSED_DATA.loc(axis=1)[:,['x','y','f','i']] #PROCESSED_DATA.swaplevel(axis=1)[['x','y','f','i']].swaplevel(axis=1)#T#.stack()
temp
pd.DataFrame(np.vstack(temp.iloc[0].values).T, columns = temp.columns) #swaplevel(axis=1)['y']

pd.DataFrame([x, y], index=['x','y']).T
'''
pass

#### Widget

In [ ]:
@solara.component
def OutputManager(process_callback, save_callback, make_fig):

    global PROCESSED_DATA, TABLE

    save_name, set_save_name = solara.use_state('default.csv')


    #if file_open_callback is None:
    #    def click_callback():
    #        pass
    #else:


    def click_callback():

        process_callback()
        #if PROCESSED_DATA != None:
        #    TABLE.data = make_summary_fig(PROCESSED_DATA)
        #else:
        #    TABLE.data = None
        make_fig()


    def save_callback_():
        save_callback(save_name)

    main = solara.VBox(children=[
        solara.Button("Process All", on_click=click_callback),
        #solara.DataFrame(PROCESSED_DATA)
        solara.InputText(label="filename (end with .csv): ", value=save_name,on_value=set_save_name),
        solara.Button("Save and Clear", on_click=save_callback_),
        TABLE
        ])

        #solara.Info(f"You selected path: {path}")
        #solara.Info(f"You opened file: {file}")
    return main

In [ ]:
@solara.component
def ProcessController():
    global PROFILE_HALF_LENGTH, PROFILE_SEGMENT_LENGTH, CONTROL_ELEMENTS

    halfLength, set_halfLength = solara.use_state(PROFILE_HALF_LENGTH)
    segLength, set_segLength = solara.use_state(PROFILE_SEGMENT_LENGTH)
    fitter, set_fitter = solara.use_state(CONTROL_ELEMENTS['fitter'])
    form, set_form = solara.use_state(CONTROL_ELEMENTS['form'])
    slope, set_slope = solara.use_state(CONTROL_ELEMENTS['m'])
    time, set_time = solara.use_state(CONTROL_ELEMENTS['t'])
    BParam, set_BParam = solara.use_state(CONTROL_ELEMENTS['B'])
    intercept, set_intercept = solara.use_state(CONTROL_ELEMENTS['c'])
    plane, set_plane = solara.use_state(CONTROL_ELEMENTS['n'])
    degree, set_degree = solara.use_state(CONTROL_ELEMENTS['deg'])
    use_plane, set_use_plane = solara.use_state(CONTROL_ELEMENTS['use_n'])

    with solara.VBox() as main:

        solara.InputFloat(label="PROFILE_HALF_LENGTH: ", value=halfLength,on_value=set_halfLength)
        solara.InputFloat(label="PROFILE_SEGMENT_LENGTH: ", value=segLength,on_value=set_segLength)
        solara.Select(label="Fitter Type: ", values = ['mullins','simple'], value=fitter,on_value=set_fitter)
        solara.Markdown('Fitter Parameters')
        if fitter =='mullins':
            solara.InputFloat(label="slope: ", value=slope,on_value=set_slope)
            solara.InputFloat(label="intercept: ", value=intercept,on_value=set_intercept)
            solara.InputFloat(label="degree: ", value=degree,on_value=set_degree)
            solara.InputFloat(label="Time: ", value=time,on_value=set_time)
            solara.InputFloat(label="B: ", value=BParam,on_value=set_BParam)
            solara.Checkbox(label="Include Fit Plane: ", value=use_plane,on_value=set_use_plane)
            if use_plane:
                solara.InputFloat(label="Fit Plane Slope: ", value=plane,on_value=set_plane)
        else:
            solara.Select(label="Fitter Form Equation: ", values = ['tanh','poly', 'sine'], value=form,on_value=set_form)
            if form == 'tanh':
                solara.InputFloat(label="slope: ", value=slope,on_value=set_slope)
                solara.InputFloat(label="intercept: ", value=intercept,on_value=set_intercept)
            else:
                solara.InputFloat(label="slope: ", value=slope,on_value=set_slope)
                solara.InputFloat(label="intercept: ", value=intercept,on_value=set_intercept)
                solara.InputInt(label="degree: ", value=degree,on_value=set_degree)


    PROFILE_HALF_LENGTH = halfLength
    PROFILE_SEGMENT_LENGTH = segLength
    CONTROL_ELEMENTS['fitter'] = fitter
    CONTROL_ELEMENTS['form'] = form
    CONTROL_ELEMENTS['m'] = slope
    CONTROL_ELEMENTS['t'] = time
    CONTROL_ELEMENTS['B'] = BParam
    CONTROL_ELEMENTS['c'] = intercept
    CONTROL_ELEMENTS['n'] = plane
    CONTROL_ELEMENTS['deg'] = int(degree)
    CONTROL_ELEMENTS['use_n'] = use_plane
    return main

    #m = 0.15, t = 3600*4, B = 3.4e-20, n = 0, c = 0, deg = 3, use_n = False


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



## Interaction

In [ ]:
om = OutputManager(process_callback=try_process, save_callback=save_data, make_fig=make_summary_fig)

In [ ]:
pc = ProcessController()

In [ ]:
@solara.component
def combinedWidget():
    global dm, fig1, fig2, fig3, fig_preview, om
    #w = FigureWidget(data=fig.data)
    mdt_plots = solara.HBox(children=[fig2, fig1])
    analysis = solara.HBox(children=[fig_preview, fig3])
    #vb = solara.VBox(children=[])

    main = solara.VBox(children = [mdt_plots,
                                   dm,
                                   analysis,
                                   pc,
                                   om])

    return main

## Reset and Complete Widget

In [ ]:
CONTROL_ELEMENTS = dict(fitter='mullins', deg=25, m=.14, t= 1000, B=10, c=0, n=0, use_n=False, form='tanh')

In [ ]:
PROCESS_PROFILES = []
PROCESS_FILE_NAME = ''

Z_DATA = None
MDTFI = None
UNIT = 'um'
SPACING = (1., 1.)
ACTIVE_LINE = {'x1':0,'x2':0,'y1':0,'y2':0}
PROFILES = []
LINES = []

PROCESSED_DATA = pd.DataFrame()

fig1 = go.FigureWidget(data=custom_surface())
fig2 = go.FigureWidget(data=custom_heatmap2D())
fig_preview = go.FigureWidget(data=plot_profile("Previewing"))
fig3 = go.FigureWidget(data=plot_profile("Processing"))
#fw = go.FigureWidget(data=fig.data)#.element()
fig2.data[0].on_click(update_point)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'''))

combinedWidget()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



<IPython.core.display.Javascript object>

Html(layout=None, style_='display: none', tag='span')

Cannot show ipywidgets in text

<lambdifygenerated-11>:2: RuntimeWarning:

invalid value encountered in double_scalars

<lambdifygenerated-11>:2: RuntimeWarning:

invalid value encountered in double_scalars



Left side of lc02.txt could not be fit, skipping inclusion in param dataframe
             left       right
            angle       angle
count    3.000000    3.000000
mean   173.808232  168.420863
std      2.084493    1.403033
min    171.401271  166.854025
25%    173.204332  167.850782
50%    175.007393  168.847539
75%    175.011712  169.204282
max    175.016031  169.561025


<frozen importlib._bootstrap>:914: ImportWarning:

APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()

<frozen importlib._bootstrap>:914: ImportWarning:

_PyDriveImportHook.find_spec() not found; falling back to find_module()

<frozen importlib._bootstrap>:914: ImportWarning:

_OpenCVImportHook.find_spec() not found; falling back to find_module()

<frozen importlib._bootstrap>:914: ImportWarning:

_BokehImportHook.find_spec() not found; falling back to find_module()

<frozen importlib._bootstrap>:914: ImportWarning:

_AltairImportHook.find_spec() not found; falling back to find_module()

<lambdifygenerated-13>:2: RuntimeWarning:

invalid value encountered in double_scalars

<lambdifygenerated-13>:2: RuntimeWarning:

invalid value encountered in double_scalars



Left side of lc02.txt could not be fit, skipping inclusion in param dataframe
             left       right
            angle       angle
count    3.000000    3.000000
mean   173.808232  168.420863
std      2.084493    1.403033
min    171.401271  166.854025
25%    173.204332  167.850782
50%    175.007393  168.847539
75%    175.011712  169.204282
max    175.016031  169.561025


In [ ]:
PROCESSED_DATA

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



left                                                         \
                 m           t          B         c       angle     m_std   
lc01.txt  0.075179  461.818323  46.019287  3.037476  171.401271  0.000602   
lc04.txt  0.043521   50.826779   9.248257  1.937564  175.016031  0.000315   
lc06.txt  0.043596   68.177634  12.431755  1.989816  175.007393  0.000730   

                                                             \
                                                          x   
lc01.txt  [0.0, 0.0040701893, 0.008140379, 0.012210567, ...   
lc04.txt  [0.0, 0.0039543696, 0.007908739, 0.011863109, ...   
lc06.txt  [0.0, 0.0036864337, 0.0073728673, 0.011059301,...   

                                                             \
                                                          y   
lc01.txt  [1.6205696, 1.6216359, 1.6220821, 1.6224937, 1...   
lc04.txt  [1.6196438, 1.6207013, 1.6221815, 1.6221815, 1...   
lc06.txt  [1.6204898, 1.6231046, 1.6236227, 1.623936, 1....   

                                                             \
                                                          f   
lc01.txt  [1.6212107572841141, 1.621516750949121, 1.6218...   
lc04.txt  [1.6213877810201633, 1.6215598782563962, 1.621...   
lc06.txt  [1.622798204213759, 1.622958919099069, 1.62311...   

                                                                right  \
                                                          i         m   
lc01.txt  [1.6212107572841141, 1.621516750949121, 1.6218...  0.115226   
lc04.txt  [1.6213877810201633, 1.6215598782563962, 1.621...  0.091350   
lc06.txt  [1.622798204213759, 1.622958919099069, 1.62311...  0.097632   

                                                                 \
                   t          B         c       angle     m_std   
lc01.txt  135.813483  15.237385  2.833323  166.854025  0.001214   
lc04.txt  437.037633  79.671350  3.570652  169.561025  0.000796   
lc06.txt -171.022276 -31.177121  2.925362  168.847539  0.001139   

                                                             \
                                                          x   
lc01.txt  [0.0, 0.004070189, 0.008140374, 0.012210567, 0...   
lc04.txt  [0.0, 0.0039543696, 0.007908739, 0.011863109, ...   
lc06.txt  [0.0, 0.0036864355, 0.0073728673, 0.011059299,...   

                                                             \
                                                          y   
lc01.txt  [1.6205696, 1.6206635, 1.6218967, 1.6228255, 1...   
lc04.txt  [1.6196438, 1.6203074, 1.6228426, 1.6256013, 1...   
lc06.txt  [1.6204898, 1.6232289, 1.6253295, 1.6255708, 1...   

                                                             \
                                                          f   
lc01.txt  [1.6207486832247395, 1.6212176756287404, 1.621...   
lc04.txt  [1.6236867533657586, 1.6240479853376844, 1.624...   
lc06.txt  [1.6236696239132151, 1.6240295380752239, 1.624...   

                                                             
                                                          i  
lc01.txt  [1.6207486832247395, 1.6212176756287404, 1.621...  
lc04.txt  [1.6236867533657586, 1.6240479853376844, 1.624...  
lc06.txt  [1.6236696239132151, 1.6240295380752239, 1.624...